# Media Outlets Activity on Facebook aggregated by Media Type

The parameters in the cell below can be adjusted to explore other media outlets and time frames.

### How to explore other media types?
The ***media*** parameter can be use to aggregate media outlets by their type. The column `subcategory` in this [this other notebook](../media.ipynb?autorun=true) show the media outlets that belong each type.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
media = 'Newspaper'
aggregation = 'week' # aggregation can be by day ('day'), week ('week') or month('month')
from_date='2017-09-01'
to_date='2018-12-31'

## Using API

In [ ]:
import pandas as pd

# Create an instance to the smm wrapper
from smm_wrapper import SMMOrganizations
smm = SMMOrganizations()

#using the api to get the data
df = smm.dv.get_organizations()
df = df[(df['category']=='media')]

# Filter the accounts by party, and valid ones (the ones that contain fb_ids)
media_df = df[(df['subcategory'].str.contains(media)) & (df['fb_ids'].notnull())] 

# make sure that accounts are not repeated, e.g. Die Zeit (Online) and Die Zeit (Allgemeine) could
# share an account if facebook but not in twitter
fbids = set(fb_id for fb_ids in media_df['fb_ids'] for fb_id in fb_ids)

# query the Social Media Monitoring API
posts_by = pd.concat(smm.dv.posts_by(facebook_user_id=fb_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for fb_id in fbids)        
comments_by = pd.concat(smm.dv.comments_by(facebook_user_id=fb_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for fb_id in fbids) 

# aggregate the posts
total_posts_by = posts_by.groupby('date').agg(
    {
        'posts': sum,
        'replies': sum, 
        'shares': sum,
        'reactions': sum,
        'likes': sum
    })
# aggregate the comments
total_comments_by = comments_by.groupby('date').agg(
    {
        'comments': sum,
        'replies': sum,
        'likes': sum
    })

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

#plot for fb posts
plotly.offline.iplot({
    "data": [go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['posts'], name='Posts', line_shape='spline'), 
             go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['replies'], name='Replies', line_shape='spline'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['shares'], name='Shares', line_shape='spline'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['reactions'], name='Reactions', line_shape='spline'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['likes'], name='Likes', line_shape='spline')], 
    "layout": go.Layout(title='Facebook posts for {}'.format(media), xaxis={'title':''}, yaxis={'title':'N'})
})
#plot for fb comments
plotly.offline.iplot({
    "data": [go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['comments'], name='Comments', line_shape='spline'), 
             go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['replies'], name='Replies', line_shape='spline'),
            go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['likes'], name='Likes', line_shape='spline')], 
    "layout": go.Layout(title='Facebook comments for {}'.format(media), xaxis={'title':''}, yaxis={'title':'N'})
})